# Chương 7: Xây dựng ứng dụng trò chuyện
## Bắt đầu nhanh với Azure OpenAI API


## Tổng quan
Notebook này được điều chỉnh từ [Kho mẫu Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), bao gồm các notebook cũng truy cập dịch vụ [OpenAI](notebook-openai.ipynb).

API OpenAI Python cũng hoạt động với Azure OpenAI, chỉ cần một vài thay đổi nhỏ. Tìm hiểu thêm về sự khác biệt tại đây: [Cách chuyển đổi giữa các endpoint OpenAI và Azure OpenAI với Python](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

Để xem thêm các ví dụ khởi động nhanh, vui lòng tham khảo [Tài liệu Khởi động nhanh Azure OpenAI chính thức](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst)


## Mục lục  

[Tổng quan](../../../../07-building-chat-applications/python)  
[Bắt đầu với Azure OpenAI Service](../../../../07-building-chat-applications/python)  
[Tạo prompt đầu tiên của bạn](../../../../07-building-chat-applications/python)  

[Tình huống sử dụng](../../../../07-building-chat-applications/python)    
[1. Tóm tắt văn bản](../../../../07-building-chat-applications/python)  
[2. Phân loại văn bản](../../../../07-building-chat-applications/python)  
[3. Tạo tên sản phẩm mới](../../../../07-building-chat-applications/python)  
[4. Tinh chỉnh bộ phân loại](../../../../07-building-chat-applications/python)  
[5. Embeddings](../../../../07-building-chat-applications/python)

[Tài liệu tham khảo](../../../../07-building-chat-applications/python)


### Bắt đầu với Dịch vụ Azure OpenAI

Khách hàng mới sẽ cần [đăng ký để được cấp quyền truy cập](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst) vào Dịch vụ Azure OpenAI.  
Sau khi được phê duyệt, khách hàng có thể đăng nhập vào cổng thông tin Azure, tạo một tài nguyên Dịch vụ Azure OpenAI và bắt đầu thử nghiệm với các mô hình thông qua studio  

[Nguồn tài liệu tuyệt vời để bắt đầu nhanh chóng](https://techcommunity.microsoft.com/blog/educatordeveloperblog/azure-openai-service-is-now-generally-available/3719177?WT.mc_id=academic-105485-koreyst)


### Tạo prompt đầu tiên của bạn  
Bài tập ngắn này sẽ giúp bạn làm quen cơ bản với việc gửi prompt cho một mô hình OpenAI để thực hiện một nhiệm vụ đơn giản là "tóm tắt".

**Các bước thực hiện**:  
1. Cài đặt thư viện OpenAI trong môi trường python của bạn  
2. Tải các thư viện hỗ trợ tiêu chuẩn và thiết lập thông tin bảo mật OpenAI thông thường cho Dịch vụ OpenAI mà bạn đã tạo  
3. Chọn một mô hình phù hợp cho nhiệm vụ của bạn  
4. Tạo một prompt đơn giản cho mô hình  
5. Gửi yêu cầu của bạn đến API của mô hình!


> [!NOTE] Bước này không cần thiết nếu chạy notebook này trên Codespaces hoặc trong Devcontainer


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

### 3. Tìm kiếm mô hình phù hợp  
Các mô hình GPT-3.5-turbo hoặc GPT-4 có khả năng hiểu và tạo ra ngôn ngữ tự nhiên. Dịch vụ này cung cấp bốn loại mô hình với các mức độ mạnh mẽ và tốc độ khác nhau, phù hợp cho từng nhiệm vụ riêng biệt.

[Azure OpenAI models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)


In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. Thiết kế Prompt  

"Điều kỳ diệu của các mô hình ngôn ngữ lớn là khi được huấn luyện để giảm thiểu lỗi dự đoán trên một lượng lớn văn bản, các mô hình này sẽ học được những khái niệm hữu ích cho việc dự đoán. Ví dụ, chúng học được các khái niệm như"(1):

* cách viết đúng chính tả
* cách sử dụng ngữ pháp
* cách diễn đạt lại ý
* cách trả lời câu hỏi
* cách trò chuyện
* cách viết bằng nhiều ngôn ngữ
* cách lập trình
* v.v.

#### Cách kiểm soát mô hình ngôn ngữ lớn  
"Trong tất cả các đầu vào của mô hình ngôn ngữ lớn, văn bản prompt là yếu tố có ảnh hưởng lớn nhất"(1).

Các mô hình ngôn ngữ lớn có thể được hướng dẫn để tạo ra đầu ra theo một số cách:

- Chỉ dẫn: Nói cho mô hình biết bạn muốn gì
- Hoàn thành: Gợi ý để mô hình hoàn thành phần đầu của nội dung bạn muốn
- Minh họa: Cho mô hình thấy bạn muốn gì, bằng cách:
- Đưa một vài ví dụ trong prompt
- Đưa hàng trăm hoặc hàng nghìn ví dụ trong bộ dữ liệu huấn luyện fine-tuning



#### Có ba nguyên tắc cơ bản để tạo prompt:

**Chỉ rõ và minh họa**. Hãy làm rõ bạn muốn gì, thông qua hướng dẫn, ví dụ, hoặc kết hợp cả hai. Nếu bạn muốn mô hình sắp xếp một danh sách theo thứ tự chữ cái hoặc phân loại một đoạn văn theo cảm xúc, hãy cho nó thấy đó là điều bạn cần.

**Cung cấp dữ liệu chất lượng**. Nếu bạn đang xây dựng một bộ phân loại hoặc muốn mô hình tuân theo một mẫu nhất định, hãy đảm bảo có đủ ví dụ. Đừng quên kiểm tra lại các ví dụ của mình — mô hình thường đủ thông minh để nhận ra lỗi chính tả cơ bản và vẫn trả lời, nhưng nó cũng có thể cho rằng đó là chủ ý và điều này có thể ảnh hưởng đến kết quả.

**Kiểm tra các thiết lập.** Các thiết lập temperature và top_p kiểm soát mức độ quyết đoán của mô hình khi tạo ra câu trả lời. Nếu bạn cần một câu trả lời mà chỉ có một đáp án đúng, hãy đặt các giá trị này thấp. Nếu bạn muốn có nhiều câu trả lời đa dạng hơn, hãy đặt chúng cao hơn. Sai lầm phổ biến nhất là nghĩ rằng các thiết lập này là nút điều chỉnh "sự thông minh" hoặc "sáng tạo" của mô hình.


Nguồn: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Tóm tắt văn bản  
#### Thử thách  
Tóm tắt văn bản bằng cách thêm 'tl;dr:' vào cuối một đoạn văn. Lưu ý cách mô hình hiểu và thực hiện nhiều nhiệm vụ mà không cần hướng dẫn bổ sung. Bạn có thể thử nghiệm với các lời nhắc mô tả hơn thay cho tl;dr để điều chỉnh hành vi của mô hình và cá nhân hóa bản tóm tắt mà bạn nhận được(3).  

Những nghiên cứu gần đây đã cho thấy những tiến bộ đáng kể trong nhiều nhiệm vụ và tiêu chuẩn NLP nhờ tiền huấn luyện trên một tập dữ liệu văn bản lớn, sau đó tinh chỉnh cho từng nhiệm vụ cụ thể. Mặc dù kiến trúc thường không phụ thuộc vào nhiệm vụ, phương pháp này vẫn cần các bộ dữ liệu tinh chỉnh riêng biệt với hàng nghìn hoặc hàng chục nghìn ví dụ. Ngược lại, con người thường có thể thực hiện một nhiệm vụ ngôn ngữ mới chỉ với vài ví dụ hoặc chỉ dẫn đơn giản – điều mà các hệ thống NLP hiện tại vẫn còn gặp khó khăn. Ở đây, chúng tôi cho thấy rằng việc mở rộng quy mô các mô hình ngôn ngữ giúp cải thiện đáng kể hiệu suất với ít ví dụ, không phụ thuộc vào nhiệm vụ, thậm chí đôi khi còn đạt mức cạnh tranh với các phương pháp tinh chỉnh tiên tiến trước đó.  

Tl;dr


# Bài tập cho nhiều trường hợp sử dụng  
1. Tóm tắt văn bản  
2. Phân loại văn bản  
3. Tạo tên sản phẩm mới  
4. Embeddings  
5. Tinh chỉnh bộ phân loại


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Phân loại Văn bản  
#### Thử thách  
Phân loại các mục vào các danh mục được cung cấp tại thời điểm suy luận. Trong ví dụ sau, chúng ta cung cấp cả các danh mục và văn bản cần phân loại trong prompt (*playground_reference).

Yêu cầu từ khách hàng: Xin chào, một trong các phím trên bàn phím laptop của tôi vừa bị hỏng gần đây và tôi cần thay thế:

Danh mục đã phân loại:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Tạo Tên Sản Phẩm Mới
#### Thử thách
Tạo tên sản phẩm từ các từ ví dụ. Ở đây, chúng tôi đưa vào phần nhắc thông tin về sản phẩm mà chúng tôi sẽ tạo tên. Chúng tôi cũng cung cấp một ví dụ tương tự để thể hiện kiểu mẫu mà chúng tôi mong muốn nhận được. Ngoài ra, chúng tôi đã đặt giá trị nhiệt độ cao để tăng tính ngẫu nhiên và nhận được các phản hồi sáng tạo hơn.

Mô tả sản phẩm: Máy làm sữa lắc tại nhà
Từ khóa gợi ý: nhanh, khỏe mạnh, nhỏ gọn.
Tên sản phẩm: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Mô tả sản phẩm: Một đôi giày có thể vừa với mọi kích cỡ chân.
Từ khóa gợi ý: linh hoạt, vừa vặn, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## Embeddings
Phần này sẽ hướng dẫn cách lấy embeddings và tìm sự tương đồng giữa các từ, câu và tài liệu. Để chạy các notebook dưới đây, bạn cần triển khai một mô hình sử dụng `text-embedding-ada-002` làm mô hình gốc và đặt tên triển khai của nó trong file .env, sử dụng biến `AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT`.


### Phân loại mô hình - Lựa chọn mô hình embedding

**Phân loại mô hình**: {family} - {capability} - {input-type} - {identifier}  

{family}     --> text-embedding  (họ embedding)  
{capability} --> ada             (tất cả các mô hình embedding khác sẽ bị ngừng sử dụng vào năm 2024)  
{input-type} --> n/a             (chỉ được chỉ định cho các mô hình tìm kiếm)  
{identifier} --> 002             (phiên bản 002)  

model = 'text-embedding-ada-002'


> [!NOTE] Bước sau đây không cần thiết nếu bạn chạy notebook này trên Codespaces hoặc trong Devcontainer


In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

## So sánh bài báo từ bộ dữ liệu tin tức hàng ngày của CNN
nguồn: https://huggingface.co/datasets/cnn_dailymail


In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# Tài liệu tham khảo  
- [Tài liệu Azure - Các mô hình Azure OpenAI](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [Ví dụ về OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)


# Để Được Hỗ Trợ Thêm  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Người đóng góp
* Louis Li



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm đối với bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
